In [1]:
# ============================
# STEP 1: Install dependencies
# ============================
!sudo apt install tesseract-ocr -y
!sudo apt install libtesseract-dev -y
!sudo apt install tesseract-ocr-ara -y  # Arabic language pack
!pip install pytesseract fitz Pillow pandas deep-translator

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-ara is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
# ============================
# STEP 2: Upload PDF
# ============================

uploaded = files.upload()  # Upload your scanned PDF here
pdf_path = list(uploaded.keys())[0]


IndexError: list index out of range

In [6]:
# ============================
# STEP 3: Extract images from PDF
# ============================
!pip install pytesseract pymupdf Pillow pandas deep-translator
from google.colab import files
import fitz  # PyMuPDF
import io
from PIL import Image
import pytesseract

doc = fitz.open("/content/file-data-CSV.pdf")
ocr_results = []

for page_num in range(len(doc)):
    page = doc[page_num]
    for img_index, img in enumerate(page.get_images(full=True)):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        # OCR in Arabic
        text = pytesseract.image_to_string(image, lang="ara")
        ocr_results.append(text)

print("✅ OCR extraction complete, found", len(ocr_results), "tables/pages.")

# ============================
# STEP 4: Parse into CSV
# ============================
import pandas as pd

rows = []
for block in ocr_results:
    for line in block.split("\n"):
        if line.strip():
            # Split columns by spaces/tabs
            cols = [c.strip() for c in line.split() if c.strip()]
            rows.append(cols)

df_ar = pd.DataFrame(rows)
df_ar.to_csv("data_arabic.csv", index=False, header=False, encoding="utf-8-sig")

print("✅ Arabic CSV saved as data_arabic.csv")

# ============================
# STEP 5: Translate to English
# ============================
from deep_translator import GoogleTranslator

def translate_cell(text):
    try:
        return GoogleTranslator(source="auto", target="en").translate(text)
    except:
        return text

df_en = df_ar.applymap(translate_cell)
df_en.to_csv("data_english.csv", index=False, header=False, encoding="utf-8-sig")

print("✅ English CSV saved as data_english.csv")

# ============================
# STEP 6: Download Results
# ============================
files.download("data_arabic.csv")
files.download("data_english.csv")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 75.4 MB/s eta 0:00:00
✅ OCR extraction complete, found 44 tables/pages.
✅ Arabic CSV saved as data_arabic.csv


/tmp/ipython-input-658450933.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_en = df_ar.applymap(translate_cell)


✅ English CSV saved as data_english.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>